In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/MyDrive/drive-download-20240115T000348Z-001/Лабораторная7"

Mounted at /content/drive
/content/drive/MyDrive/drive-download-20240115T000348Z-001/Лабораторная7


In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import MeanShift, estimate_bandwidth
from google.colab.patches import cv2_imshow

In [3]:
def rotate_image(image, angle):
    # Получение центра изображения
    height, width = image.shape[:2]
    center = (width / 2, height / 2)

    # Создание матрицы преобразования поворота
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)

    # Применение матрицы преобразования к изображению
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))

    return rotated_image

In [4]:
# image = cv2.imread('../images/pear.png')
# template = cv2.imread('../images/pear_template.png')
# image = cv2.imread('mario.png')
# template = cv2.imread('tmplenemy.png')
image = cv2.imread('image.jpg')
template = cv2.imread('template.jpg')

cv2_imshow(image)
cv2_imshow(template)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
def match_rotated_template(template, image):
    # Преобразование изображения и шаблона в оттенки серого
    image = image.copy()
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

    # Получение размеров шаблона
    h, w = gray_template.shape

    # Задание диапазона углов поворота
    angles = np.arange(0, 360)

    # Список для хранения координат всех найденных объектов
    all_locations = []

    # Итерация по углам поворота
    for angle in angles:
        # Получение повернутого шаблона
        rotated_template = rotate_image(gray_template, angle)

        # Нахождение совпадений с использованием метода кросс-корреляции
        result = cv2.matchTemplate(gray_image, rotated_template, cv2.TM_CCOEFF_NORMED)

        # Нахождение координат максимального значения совпадения
        locations = np.where(result >= 0.6)
        locations = list(zip(*locations[::-1]))

        # Добавление найденных координат в общий список
        for loc in locations:
            all_locations.append((loc[0] + w // 2, loc[1] + h // 2))

    # Отмечаем все найденные объекты на изображении
    for loc in all_locations:
        cv2.rectangle(image, (loc[0] - w // 2, loc[1] - h // 2), (loc[0] + w // 2, loc[1] + h // 2), (0, 255, 0), 2)

    # Отрисовка найденных объектов на изображении
    cv2_imshow(image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Вызов функции сопоставления
match_rotated_template(template, image)

In [ ]:

scale = 1.1
scBr = 0.9

gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

template_scale = cv2.resize(np.uint8(0.9*cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)),
           (int(template.shape[1]*scale), int(template.shape[0]*scale)), interpolation = cv2.INTER_AREA)

sift = cv2.SIFT_create()

k_1, des_1 = sift.detectAndCompute(gray_image, None)
k_2, des_2 = sift.detectAndCompute(template_scale, None)

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L1, crossCheck=True)

In [ ]:
matches = sorted(bf.match(des_1, des_2), key=lambda x: x.distance)

In [ ]:
bf = cv2.BFMatcher(cv2.NORM_L1)
matches = bf.knnMatch(des_1, des_2, k=2)

good = []
for m, n in matches:
    if m.distance < 0.5 * n.distance:
        good.append([m])


In [ ]:
points = np.array([(0, 0)])
for i in good:
    points = np.append(points, [k_1[i[0].queryIdx].pt], axis=0)

points = points[1:len(points)]

bandwidth = estimate_bandwidth(points, quantile=0.1)
ms = MeanShift(bandwidth=bandwidth or None, bin_seeding=False, cluster_all=False)
cl = ms.fit_predict(points)

labels_unique = np.unique(ms.labels_)
kp = []
for i in labels_unique:
    kp.append(points[cl==i])

cen = []
for i in kp:
    if len(i)>8:
        cen.append(np.mean(i, axis=0).astype(np.uint16))

plot_img = image.copy()
h, w = template_scale.shape

for pt in cen:
    cv2.rectangle(plot_img, (pt[0] - w, pt[1] - h),(pt[0] + w, pt[1] + h),(0,255,255), 8)

cv2_imshow(plot_img)